<a href="https://colab.research.google.com/github/anusornc/AI-resources/blob/master/pineapple.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
# Initialising the CNN
classifier = Sequential()
# Step 1 - Convolution
classifier.add(Conv2D(16, (5, 5), input_shape = (196, 196, 3), activation = 'relu'))
# Step 2 - Pooling
classifier.add(MaxPooling2D(pool_size = (2, 2)))
# Adding a second convolutional layer
classifier.add(Conv2D(32, (5, 5), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))
# Step 3 - Flattening
classifier.add(Flatten())
# Step 4 - Full connection
classifier.add(Dense(units = 128, activation = 'relu'))
classifier.add(Dense(units = 3, activation = 'softmax'))
# Compiling the CNN
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [11]:
# Part 2 - Fitting the CNN to the images
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255,
                    shear_range = 0.2,
                    zoom_range = 0.2,
                    horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('pineapple-dataset/training_set',
                                target_size = (196, 196), batch_size = 10, class_mode = 'categorical',color_mode="rgb")
test_set = test_datagen.flow_from_directory('pineapple-dataset/test_set',
                                target_size = (196, 196), batch_size = 5, class_mode = 'categorical',color_mode="rgb")

classifier.fit_generator(training_set,steps_per_epoch = 8000,epochs = 2,
                            validation_data = test_set, validation_steps = 2000)


Found 102 images belonging to 3 classes.
Found 39 images belonging to 3 classes.
Epoch 1/2
8000/8000 [==============================] - 10068s 1s/step - loss: 5.7797 - acc: 0.6410 - val_loss: 12.3985 - val_acc: 0.2308
Epoch 2/2
8000/8000 [==============================] - 9968s 1s/step - loss: 9.8137 - acc: 0.3911 - val_loss: 12.3985 - val_acc: 0.2308


In [18]:
import numpy as np
from keras.preprocessing import image
test_image = image.load_img('pineapple-dataset/single_prediction/U1.jpg', target_size = (196, 196))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = classifier.predict(test_image)
training_set.class_indices
print(result,training_set.class_indices)

[[0. 0. 1.]] {'fully-ripe': 0, 'partial-ripe': 1, 'unripe': 2}


In [10]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [1]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 110842 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.1-0ubuntu3~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.1-0ubuntu3~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.1-0ubuntu3~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=ht

In [2]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate


KeyboardInterrupt: ignored

In [0]:
!mkdir -p pineapple-dataset/single_prediction

In [0]:
!mkdir -p pineapple-dataset/training_set/unripe pineapple-dataset/training_set/partial-ripe pineapple-dataset/training_set/fully-ripe

In [25]:
from sklearn.metrics import classification_report, confusion_matrix

Y_pred = classifier.predict_generator(test_set, 39 // 5+1)
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
print(confusion_matrix(test_set.classes, y_pred))
print('Classification Report')
target_names = ['fully-ripe' , 'partial-ripe' , 'unripe']
print(classification_report(test_set.classes, y_pred, target_names=target_names))

Confusion Matrix
[[ 0  0 23]
 [ 0  0  7]
 [ 0  0  9]]
Classification Report
              precision    recall  f1-score   support

  fully-ripe       0.00      0.00      0.00        23
partial-ripe       0.00      0.00      0.00         7
      unripe       0.23      1.00      0.38         9

   micro avg       0.23      0.23      0.23        39
   macro avg       0.08      0.33      0.12        39
weighted avg       0.05      0.23      0.09        39



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
